In this notebook the first treatments to the data of the uber2012 dataset will be carried out,

In the end we will generate a new file with the treatment performed to share them and not have to run the entire treatment when we want to analyze the data later

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from geopandas import GeoDataFrame

In [4]:
poligonos = gpd.read_file("./data/ny_nj_poligonos.geojson")

In [5]:
yellow12 = pd.read_csv("data/yellow12_1.csv")

In [6]:
print(yellow12.shape)
yellow12.head(2)

(7800000, 7)


,Unnamed: 0,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Start_Lon,Start_Lat,End_Lon,End_Lat
0,0,2012-01-19 08:45:06,2012-01-19 08:50:52,-74.010669,40.715217,-74.005392,40.711362
1,1,2012-01-05 00:42:00,2012-01-05 00:46:00,-74.002933,40.718197,-74.013570,40.705077


In [7]:
yellow12 = yellow12.drop(["Unnamed: 0"],axis=1)
yellow12 = yellow12.rename(columns={"Trip_Pickup_DateTime":"pickup_datetime",
                                    "Trip_Dropoff_DateTime":"dropoff_datetime",
                                    "Start_Lon":"pickup_longitude",
                                    "Start_Lat":"pickup_latitude",
                                    "End_Lon":"dropoff_longitude",
                                    "End_Lat":"dropoff_latitude"})

In [8]:
# Geometry Green pickup coordenates
geometry_yellow12_pick = [Point(xy) for xy in zip(yellow12.pickup_longitude, yellow12.pickup_latitude)]
crs = {'init': 'epsg:4326'}
geo_yellow12 = GeoDataFrame(yellow12, crs=crs, geometry=geometry_yellow12_pick)

In [9]:
yellow12_nta = gpd.sjoin(geo_yellow12, poligonos[["boro_name","ntacode","ntaname","geometry"]], how="left", op='intersects')

In [11]:
yellow12_nta = yellow12_nta.rename(columns={"boro_name":"pickup_borough","ntacode":"pickup_ntacode","ntaname":"pickup_ntaname","geometry":"p_geometry"})
yellow12_nta = yellow12_nta.loc[~yellow12_nta.index.duplicated(keep='first')]
yellow12_nta = yellow12_nta.drop(["index_right"],axis=1)

In [12]:
print(yellow12_nta.shape)
yellow12_nta.head()

(7800000, 10)


,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,p_geometry,pickup_borough,pickup_ntacode,pickup_ntaname
0,2012-01-19 08:45:06,2012-01-19 08:50:52,-74.010669,40.715217,-74.005392,40.711362,POINT (-74.01067 40.71522),Manhattan,MN24,SoHo-TriBeCa-Civic Center-Little Italy
1,2012-01-05 00:42:00,2012-01-05 00:46:00,-74.002933,40.718197,-74.013570,40.705077,POINT (-74.00293 40.71820),Manhattan,MN24,SoHo-TriBeCa-Civic Center-Little Italy
2,2012-01-08 12:12:00,2012-01-08 12:15:00,-73.961967,40.773578,-73.954072,40.778792,POINT (-73.96197 40.77358),Manhattan,MN40,Upper East Side-Carnegie Hill
3,2012-01-04 21:19:13,2012-01-04 21:28:51,-73.987095,40.720450,-73.985948,40.743825,POINT (-73.98709 40.72045),Manhattan,MN27,Chinatown
4,2012-01-16 15:20:48,2012-01-16 15:56:26,-73.856200,40.759600,-73.604600,41.088200,POINT (-73.85620 40.75960),Queens,QN27,East Elmhurst


In [13]:
yellow12_nta.to_csv("data/yellow12_nta.csv",index=False)

In [14]:
prueba = pd.read_csv("data/yellow12_nta.csv")

In [15]:
print(prueba.shape)
prueba.head()

(7800000, 10)


,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,p_geometry,pickup_borough,pickup_ntacode,pickup_ntaname
0,2012-01-19 08:45:06,2012-01-19 08:50:52,-74.010669,40.715217,-74.005392,40.711362,POINT (-74.01066899999998 40.715217),Manhattan,MN24,SoHo-TriBeCa-Civic Center-Little Italy
1,2012-01-05 00:42:00,2012-01-05 00:46:00,-74.002933,40.718197,-74.013570,40.705077,POINT (-74.002933 40.718197),Manhattan,MN24,SoHo-TriBeCa-Civic Center-Little Italy
2,2012-01-08 12:12:00,2012-01-08 12:15:00,-73.961967,40.773578,-73.954072,40.778792,POINT (-73.961967 40.773578),Manhattan,MN40,Upper East Side-Carnegie Hill
3,2012-01-04 21:19:13,2012-01-04 21:28:51,-73.987095,40.720450,-73.985948,40.743825,POINT (-73.98709499999998 40.72045),Manhattan,MN27,Chinatown
4,2012-01-16 15:20:48,2012-01-16 15:56:26,-73.856200,40.759600,-73.604600,41.088200,POINT (-73.8562 40.7596),Queens,QN27,East Elmhurst
